<a href="https://colab.research.google.com/github/aglucaci/Bioinformatics-Studio/blob/main/notebooks/Bioinformatics_Studio_NG86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imports
import os
import sys
!pip install biopython
from Bio import SeqIO
import pandas as pd
from google.colab import files
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 6.9 MB/s eta 0:00:00


In [3]:
# Get data (Multiple sequence alignment)
!wget https://raw.githubusercontent.com/aglucaci/Bioinformatics-Studio/main/data/ACE2_codons.SA.fasta -O ACE2.fasta
!ls

--2023-08-07 13:03:15--  https://raw.githubusercontent.com/aglucaci/Bioinformatics-Studio/main/data/ACE2_codons.SA.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96190 (94K) [text/plain]
Saving to: ‘ACE2.fasta’

ACE2.fasta          100%[===================>]  93.94K  --.-KB/s    in 0.02s   

2023-08-07 13:03:15 (5.78 MB/s) - ‘ACE2.fasta’ saved [96190/96190]

ACE2.fasta  sample_data


In [4]:
# Declares
results = {}
fasta   = "ACE2.fasta"

In [5]:
# Standard codon table, will need to code in the alternatives

CodonTable = {'A': ['GCA', 'GCC', 'GCG', 'GCT'],
              'C': ['TGC', 'TGT'],
              'D': ['GAC', 'GAT'],
              'E': ['GAA', 'GAG'],
              'F': ['TTC', 'TTT'],
              'G': ['GGA', 'GGC', 'GGG', 'GGT'],
              'H': ['CAC', 'CAT'],
              'I': ['ATA', 'ATC', 'ATT'],
              'K': ['AAA', 'AAG'],
              'L': ['CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'],
              'M': ['ATG'],
              'N': ['AAC', 'AAT'],
              'P': ['CCA', 'CCC', 'CCG', 'CCT'],
              'Q': ['CAA', 'CAG'],
              'R': ['AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGT'],
              'S': ['AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT'],
              'T': ['ACA', 'ACC', 'ACG', 'ACT'],
              'V': ['GTA', 'GTC', 'GTG', 'GTT'],
              'W': ['TGG'],
              'Y': ['TAC', 'TAT'],
              'Stop': ['TAA', 'TGA', 'TAG']
             }

In [39]:
# Helper function

def diff_counter(from_codon, to_codon):
    count = 0
    #counts
    if from_codon[0] != to_codon[0]:
        count += 1
    if from_codon[1] != to_codon[1]:
        count += 1
    if from_codon[2] != to_codon[2]:
        count += 1

    return count
    """
    #assign mutation type
    if count == 1:
        #change_types["SH"] += counter
        return "SH"
    if count == 2:
        #change_types["TH"] += counter
        return "DH"
    if count == 3:
        #change_types["TH"] += counter
        return "TH"
    %reload_ext
    """
#end method

def CalculateENES(Codon, CodonTable):
  """
   Input: A single codon e.g. ATG, and the codon table
   Output: EN and ES values
   Description:
   ---------------------------------------------------
   Loop through codon positions
   Mutating each position
   ES is defined by:
    We denote by 5 the fraction of synonymous changes at the ith position of a given codon(i=1,2,3).
   and EN = 3 - ES
  """
  if len(Codon) != 3:
    return -1, -1
  #end if

  EN = 0
  ES = 0
  F = [0, 0, 0] # Synonymous frequencies
  for i in range(3):            # For each position in the codon
    NTs = ['T', 'C', 'G', 'A']
    SynonymousChanges = 0
    for nt in NTs:
      if Codon[i].upper() != nt:
        NewCodon = list(Codon)
        NewCodon[i] = nt
        NewCodon = "".join(NewCodon)
        Change = [Codon[i], NewCodon[i]]
        # Is it synonymous?
        for AA in CodonTable.keys():
          if Codon in CodonTable[AA] and NewCodon in CodonTable[AA]:
            print("(Position: " + str(i+1) + ")", Codon, NewCodon, AA, CodonTable[AA], Change)
            SynonymousChanges += 1
        #end for
      #end if
    #end inner for
    F[i] = SynonymousChanges / 3
  #end outer for
  #print(F)
  ES = sum(F)
  return 3 - ES, ES
#end method

def Evaluate(CodonPair):
  NTs = ['T', 'C', 'G', 'A']
  # Given a pair of codons which correspond to a site in an alignment of two homologous sequences
  # We now compute the number of synonymous sites (s) and the number of non- synonymous sites (n) for each codon, considering the above property of codon changes.
  s = 0
  n = 0
  #for nt_site in codon:
  #  f = [0, 0, 0]
  #  for i in range(2):
  #    f[i] =

  #return s, 3 - s
  return 0, 0, 0, 0
#end method

def NG86(SeqPair):
  # loop over the pair of sequences
  # make sure they are the same size
  seq1 = str(SeqPair[0].seq)
  seq2 = str(SeqPair[1].seq)

  if not len(seq1) == len(seq2):
    return -0.1 # Soft error, not multiple of 3
  #end if

  start = 0
  while start < len(seq1) - 2:
    codon1 = seq1[start: start + 3]
    codon2 = seq2[start: start + 3]
    #print(codon1, codon2)
    #print(pair[0].id, codon1)
    #print(pair[1].id, codon2)
    start += 1

    if codon1 == "---" or codon2 == "---":
      continue
    elif "N" in codon1 or "N" in codon2:
      return -0.2 # Ambiguous codon
    else:
      pass
    #end if

    # Done with quality control, start comparison
    # Initialize values
    N = 1
    S = 1
    EN = 1
    ES = 1

    N, S, EN, ES = Evaluate((codon1, codon2))

    dN = (N / S)
    dS = (EN / ES)
  #end while

  #print()
  #sys.exit(1)
  return dN / dS
#end method

In [38]:


#EN, ES = CalculateENES("ATG", CodonTable)
EN, ES = CalculateENES("TTA", CodonTable)
EN, ES

(3.0, 0.0)

In [45]:
# Load alignment
# and do Pairwise comparison

with open(fasta, "r") as fh:
  for n, record in enumerate(SeqIO.parse(fasta, "fasta")):
    results[record.id] = {}
    for n2, record2 in enumerate(SeqIO.parse(fasta, "fasta")):
      pair = (record, record2)
      if record.id != record2.id:
        #results[record.id] = {record2.id: NG86(pair)}
        #results[count] = {record.id: {record2.id: NG86(pair)}}
        results[record.id].update({record2.id: NG86(pair)})
      #end if
    #end inner for
  #end outer for
  fh.close()
#end with

In [27]:
df = pd.DataFrame.from_dict(results)
df

,XM_017512376_2_PREDICTED_CEBUS_IMITATOR_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,NM_001371415_1_HOMO_SAPIENS_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_1_MRNA_1,XM_016942979_2_PREDICTED_PAN_TROGLODYTES_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,NM_001135696_1_MACACA_MULATTA_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,NM_001131132_2_PONGO_ABELII_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_008988993_4_PREDICTED_CALLITHRIX_JACCHUS_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_003261084_3_PREDICTED_NOMASCUS_LEUCOGENYS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_003791864_2_PREDICTED_OTOLEMUR_GARNETTII_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_008974180_2_PREDICTED_PAN_PANISCUS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,XM_021933040_1_PREDICTED_PAPIO_ANUBIS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,...,XM_020285237_1_PREDICTED_MICROCEBUS_MURINUS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_017888580_1_PREDICTED_RHINOPITHECUS_BIETI_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,XM_023199053_2_PREDICTED_PILIOCOLOBUS_TEPHROSCELES_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_025372062_1_PREDICTED_THEROPITHECUS_GELADA_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_032285963_1_PREDICTED_SAPAJUS_APELLA_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,XM_032756617_1_PREDICTED_HYLOBATES_MOLOCH_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_045538336_1_PREDICTED_LEMUR_CATTA_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_050777627_1_PREDICTED_MACACA_THIBETANA_THIBETANA_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,XM_053580366_1_PREDICTED_NYCTICEBUS_COUCANG_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,XM_054471966_1_PREDICTED_PONGO_PYGMAEUS_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1
NM_001371415_1_HOMO_SAPIENS_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_1_MRNA_1,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_016942979_2_PREDICTED_PAN_TROGLODYTES_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
NM_001135696_1_MACACA_MULATTA_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
NM_001131132_2_PONGO_ABELII_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_008988993_4_PREDICTED_CALLITHRIX_JACCHUS_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_MRNA_1,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_003261084_3_PREDICTED_NOMASCUS_LEUCOGENYS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_003791864_2_PREDICTED_OTOLEMUR_GARNETTII_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_MRNA_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_008974180_2_PREDICTED_PAN_PANISCUS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_021933040_1_PREDICTED_PAPIO_ANUBIS_ANGIOTENSIN_I_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
XM_010336623_2_PREDICTED_SAIMIRI_BOLIVIENSIS_BOLIVIENSIS_ANGIOTENSIN_CONVERTING_ENZYME_2_ACE2_TRANSCRIPT_VARIANT_X1_MRNA_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [22]:
# Save to CSV
output_csv = fasta + ".csv"
df.to_csv(output_csv)


In [ ]:
# Download to local
files.download(output_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# End of file